In [1]:
# Add the 'interfaces' directory to the Python path so we can import alpaca_spot.py
import sys
sys.path.append('../interfaces')

# Import the AlpacaSpot wrapper class
from alpaca_spot import AlpacaSpot

# Initialize the AlpacaSpot client using credentials from config.py
alpaca = AlpacaSpot()

# Set the symbol and timeframe to use for testing
symbol = "ETH/USD"
timeframe = "1h"
amount = 0.0001

In [2]:
# Fetch the latest traded price for the selected symbol
ticker = alpaca.fetch_ticker(symbol)
print(f"Ticker for {symbol}: {ticker}")

Ticker for ETH/USD: {'symbol': 'ETH/USD', 'price': 4408.63}


In [3]:
# Retrieve the cash balance and any open positions
balance = alpaca.fetch_balance()
print("Balance:")
balance

Balance:


{'USD': {'total': 92664.9},
 'AAAU': {'total': 32.975},
 'AFL': {'total': 105.38},
 'AON': {'total': 367.01},
 'BNDX': {'total': 49.14},
 'BTCUSD': {'total': 31819.953117},
 'HCA': {'total': 395.79},
 'KMX': {'total': 57.76},
 'ORCL': {'total': 248.28},
 'ORLY': {'total': 101.54},
 'PSLV': {'total': 12.9},
 'ROST': {'total': 146.94},
 'TJX': {'total': 132.62},
 'URA': {'total': 39.01},
 'VRSK': {'total': 267.86},
 'ZTS': {'total': 154.11}}

In [4]:
# Retrieve the most recent candlestick (OHLCV) data for the symbol and timeframe
ohlcv = alpaca.fetch_recent_ohlcv(symbol, timeframe, limit=20)
print("Latest OHLCV data:")
ohlcv.tail()

Latest OHLCV data:


,open,high,low,close,volume
timestamp,,,,,
2025-08-16 10:00:00+00:00,4404.715,4416.585,4378.255,4405.995,3.554821


In [5]:
# Cancel any existing open orders for this symbol (cleanup before testing)
alpaca.cancel_all_orders(symbol)
print(f"All open orders for {symbol} cancelled.")

All open orders for ETH/USD cancelled.


In [6]:
# Place a buy limit order at 90% of the current price (should not fill)
# Useful to test order placement without risking real trades
limit_price = round(ticker['price'] * 0.9, 2)
amount = 0.1  # Very small quantity for safety

try:
    order = alpaca.place_order(symbol, "buy", amount, "limit", limit_price)
    print("Limit order placed:")
    print(f"Order ID: {order.id}, Status: {order.status}, Type: {order.order_type}")
except Exception as e:
    print(f"Error placing limit order: {e}")

Limit order placed:
Order ID: c2dff72e-5baa-4bec-9e32-1cef4ae89bf5, Status: OrderStatus.PENDING_NEW, Type: OrderType.LIMIT


In [7]:
# Fetch and display all currently open orders for the symbol
orders = alpaca.fetch_open_orders(symbol)
print(f"Open Orders for {symbol}:")
for order in orders:
    print(f"  - ID: {order.id}, Qty: {order.qty}, Status: {order.status}, Type: {order.order_type}, Side: {order.side}")

Open Orders for ETH/USD:
  - ID: c2dff72e-5baa-4bec-9e32-1cef4ae89bf5, Qty: 0.1, Status: OrderStatus.NEW, Type: OrderType.LIMIT, Side: OrderSide.BUY


In [8]:
# Final cleanup: cancel all open orders to leave the account clean
alpaca.cancel_all_orders(symbol)
print("Final cleanup complete.")

Final cleanup complete.


In [9]:
# ✅ Parameters
symbol = "BTC/USD"
amount = 0.0001  # Small test amount

# Buy before selling
position = alpaca.fetch_position(symbol)
qty_available = float(position.qty) if position else 0.0

if qty_available < amount:
    print(f"Not enough BTC ({qty_available} available). Buying 0.0001 BTC first...")
    buy_order = alpaca.place_order(symbol, "buy", amount, "market")
    print("Buy order placed. Waiting for fill...")
    position = alpaca.wait_for_filled_position(symbol)
    if not position:
        raise RuntimeError("Buy order not reflected in positions after waiting. Exiting.")

# Now place the sell
print(f"Proceeding to sell 0.0001 BTC...")
sell_order = alpaca.place_order(symbol, "sell", amount, "market")
print("Sell order placed:", sell_order)


Proceeding to sell 0.0001 BTC...
Sell order placed: id=UUID('3b36ca66-1215-4db6-a5de-f61c837329a8') client_order_id='d366c9e3-2b0e-41c3-8169-6135e9ae6a62' created_at=datetime.datetime(2025, 8, 16, 11, 0, 49, 711203, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 8, 16, 11, 0, 49, 712402, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2025, 8, 16, 11, 0, 49, 711203, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None expires_at=datetime.datetime(2025, 11, 13, 21, 0, tzinfo=TzInfo(UTC)) canceled_at=None failed_at=None replaced_at=None replaced_by=None replaces=None asset_id=UUID('276e2673-764b-4ab6-a611-caf665ca6340') symbol='BTC/USD' asset_class=<AssetClass.CRYPTO: 'crypto'> notional=None qty='0.0001' filled_qty='0' filled_avg_price=None order_class=<OrderClass.SIMPLE: 'simple'> order_type=<OrderType.MARKET: 'market'> type=<OrderType.MARKET: 'market'> side=<OrderSide.SELL: 'sell'> time_in_force=<TimeInForce.GTC: 'gtc'> limit_price=None stop_price=None status=<OrderStatus.PEND

In [12]:
# 📦 Test placing different types of orders using AlpacaSpot wrapper
from alpaca_spot import AlpacaSpot
import time

alpaca = AlpacaSpot()
symbol = "BTC/USD"
qty = 0.01  # Small amount for testing

# 💰 Market Order - Buy
print("📈 Placing Market Buy Order")
market_order = alpaca.place_order(symbol, "buy", qty, "market")
print("✅ Market Buy Order Placed:", market_order)

time.sleep(3)  # Wait a moment to avoid order conflict

# 📉 Market Order - Sell
print("📉 Placing Market Sell Order")
market_sell_order = alpaca.place_order(symbol, "sell", qty, "market")
print("✅ Market Sell Order Placed:", market_sell_order)

time.sleep(3)

# 💸 Limit Order - Buy (place well below current price to avoid immediate fill)
limit_price = 10000  # Unlikely to be filled, for test
print("⏳ Placing Limit Buy Order at $10,000")
limit_order = alpaca.place_order(symbol, "buy", qty, "limit", limit_price)
print("✅ Limit Buy Order Placed:", limit_order)

time.sleep(3)

'''
# Alpaca Crypto Does Not Currently Allow Stop Loss
# 🛑 Stop-Loss Order - Sell (place above current price to avoid trigger)
stop_price = 130000  # Unlikely to be triggered, for test
print(f"🛑 Placing Stop-Loss Sell Order at ${stop_price}")
stop_order = alpaca.place_order(symbol, "sell", qty, order_type="stop", stop_price=stop_price)
print("✅ Stop-Loss Sell Order Placed:", stop_order)
'''

📈 Placing Market Buy Order
✅ Market Buy Order Placed: id=UUID('2a6562b6-fd4e-4867-b574-fd4443180ea4') client_order_id='7e85d0b2-d7db-4bd0-a9b2-0ae009d17f0c' created_at=datetime.datetime(2025, 8, 16, 11, 17, 8, 577670, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 8, 16, 11, 17, 8, 578879, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2025, 8, 16, 11, 17, 8, 577670, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None expires_at=datetime.datetime(2025, 11, 13, 21, 0, tzinfo=TzInfo(UTC)) canceled_at=None failed_at=None replaced_at=None replaced_by=None replaces=None asset_id=UUID('276e2673-764b-4ab6-a611-caf665ca6340') symbol='BTC/USD' asset_class=<AssetClass.CRYPTO: 'crypto'> notional=None qty='0.01' filled_qty='0' filled_avg_price=None order_class=<OrderClass.SIMPLE: 'simple'> order_type=<OrderType.MARKET: 'market'> type=<OrderType.MARKET: 'market'> side=<OrderSide.BUY: 'buy'> time_in_force=<TimeInForce.GTC: 'gtc'> limit_price=None stop_price=None status=<OrderStatus.PENDIN

'\n# Alpaca Crypto Does Not Currently Allow Stop Loss\n# 🛑 Stop-Loss Order - Sell (place above current price to avoid trigger)\nstop_price = 130000  # Unlikely to be triggered, for test\nprint(f"🛑 Placing Stop-Loss Sell Order at ${stop_price}")\nstop_order = alpaca.place_order(symbol, "sell", qty, order_type="stop", stop_price=stop_price)\nprint("✅ Stop-Loss Sell Order Placed:", stop_order)\n'